In [3]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTShadowClient

# Configure parameters.
SHADOW_CLIENT = "ShadowClient2_A0195017E"
HOST_NAME = "a3okd7abcwvnh4-ats.iot.ap-southeast-1.amazonaws.com"
ROOT_CA = "AmazonRootCA1.pem.txt"
PRIVATE_KEY = "3d36242fd5-private.pem.key"
CERT_FILE = "3d36242fd5-certificate.pem.crt"
SHADOW_HANDLER = "thing2_A0195017E"

# Automatically called whenever the shadow is updated.
def myShadowUpdateCallback(payload, responseStatus, token):
    print('UPDATE: $aws/things/' + SHADOW_HANDLER + '/shadow/update/#')
    print("payload = " + payload)
    print("responseStatus:" + responseStatus)
    print("token = " + token)

# Create, configure, and connect a shadow client.
myShadowClient = AWSIoTMQTTShadowClient(SHADOW_CLIENT)
myShadowClient.configureEndpoint(HOST_NAME, 8883)
myShadowClient.configureCredentials(ROOT_CA, PRIVATE_KEY,CERT_FILE)
myShadowClient.configureConnectDisconnectTimeout(10)
myShadowClient.configureMQTTOperationTimeout(5)
myShadowClient.connect()

# Create a programmatic representation of the shadow.
myDeviceShadow = myShadowClient.createShadowHandlerWithName(SHADOW_HANDLER, True)


import pandas as pd
import time, datetime
import json

#load & overwrite 
file = pd.read_csv('train_FD002.txt', delim_whitespace = True, header = None)
sensor=[]
for i in range(21):
    sensor[i]='sensor'+ str(i+1)
file.columns = ['id', 'cycle', 'os1', 'os2', 'os3'] + sensor
file['id'] = file['id'].map(lambda x: 'FD002_'+str(x))
file['Matric No.']='A0195017E'

for i in range(100):
    msg=file.loc[[i]]
    msg['timestamp']='UTC '+str(datetime.datetime.utcnow())
    msg.index=["reported"]
    msg=msg.to_json(orient='index')
    msg='{"state":'+msg+'}'
    myDeviceShadow.shadowUpdate(msg,myShadowUpdateCallback, 5)
    time.sleep(10)

OSError: 3d36242fd5-certificate.pem.crt: No such file or directory